In [0]:
# Import dependencies

import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import csv

from google.colab import files

In [2]:
#Set hyperparameters

num_classes = 10     # number of output classes discrete range [0,9]
num_epochs = 100     # number of times the entire dataset is presented to the model
batch_size = 64      # size of input data took for one iteration
lr = 1e-3            # learning rate
weight_decay = 1e-4  # regularization parameter

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

cuda:0


In [3]:
# Download CIFAR-10 and set up dataloaders

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010))])

train_data = dsets.CIFAR10(root='./data',
                           train=True,
                           transform=transform, download=True)

test_data = dsets.CIFAR10(root='./data', train=False,
                          transform=transform, download=True)

train_gen = torch.utils.data.DataLoader(dataset=train_data,
                                        batch_size=batch_size,
                                        shuffle=True, num_workers=2)

test_gen = torch.utils.data.DataLoader(dataset=test_data,
                                       batch_size=batch_size, 
                                       shuffle=False, num_workers=2)

0it [00:00, ?it/s]

170500096it [00:06, 26539724.85it/s]                               


Files already downloaded and verified


In [0]:
# Define the model

class Net(nn.Module):
    def __init__(self, num_classes):
        super(Net, self).__init__()
        
        self.conv_layer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.2),

            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.3),
            
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3, stride=1, padding=1),
            nn.ReLU(inplace=True),
            nn.BatchNorm2d(128),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout2d(p=0.4)
        )
        

        self.fc = nn.Linear(128*4*4, num_classes)


        
    def forward(self, x):
        # conv layer
        x = self.conv_layer(x)
        # flatten
        x = x.view(x.size(0), -1)
        # fc   
        x = self.fc(x)

        return x
      

In [15]:
# Build the model

net = Net(num_classes)
net.to(device)

Net(
  (conv_layer): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU(inplace)
    (5): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (7): Dropout2d(p=0.2)
    (8): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU(inplace)
    (11): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (12): ReLU(inplace)
    (13): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (14): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (15): Dropout2d(p

In [0]:
# Define loss function & optimizer

loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=lr, 
                            weight_decay=weight_decay)

In [17]:
# Train model

net.train()
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_gen):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = net(images)

        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()

        if (i+1) % 100 == 0:
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                  %(epoch+1, num_epochs, i+1, len(train_data)//batch_size, 
                    loss.item()))

Epoch [1/100], Step [100/781], Loss: 1.7074
Epoch [1/100], Step [200/781], Loss: 1.5957
Epoch [1/100], Step [300/781], Loss: 1.5788
Epoch [1/100], Step [400/781], Loss: 1.3265
Epoch [1/100], Step [500/781], Loss: 1.3113
Epoch [1/100], Step [600/781], Loss: 1.2482
Epoch [1/100], Step [700/781], Loss: 1.2045
Epoch [2/100], Step [100/781], Loss: 1.0107
Epoch [2/100], Step [200/781], Loss: 0.9745
Epoch [2/100], Step [300/781], Loss: 0.9252
Epoch [2/100], Step [400/781], Loss: 0.8418
Epoch [2/100], Step [500/781], Loss: 0.9159
Epoch [2/100], Step [600/781], Loss: 1.0478
Epoch [2/100], Step [700/781], Loss: 0.8980
Epoch [3/100], Step [100/781], Loss: 0.8531
Epoch [3/100], Step [200/781], Loss: 0.7905
Epoch [3/100], Step [300/781], Loss: 0.9442
Epoch [3/100], Step [400/781], Loss: 0.8169
Epoch [3/100], Step [500/781], Loss: 0.8185
Epoch [3/100], Step [600/781], Loss: 0.8521
Epoch [3/100], Step [700/781], Loss: 0.6792
Epoch [4/100], Step [100/781], Loss: 0.8388
Epoch [4/100], Step [200/781], L

In [29]:
# Accuracy on test set

import numpy as np

classes = ['airplane', 'automobile', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck']

test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
net.eval()

for images, labels in test_gen:
    if torch.cuda.is_available():
        images, labels = images.to(device), labels.to(device)
        
    outputs = net(images)
  
    loss = loss_function(outputs, labels)
    test_loss += loss.item()*images.size(0)
  
    _, pred = torch.max(outputs, 1)    
  
    correct_tensor = pred.eq(labels.data.view_as(pred))
    correct = np.squeeze(correct_tensor.numpy()) if not torch.cuda.is_available() else np.squeeze(correct_tensor.cpu().numpy())
  
    for i in range(16):
        label = labels.data[i]
        class_correct[label] += correct[i].item()
        class_total[label] += 1

test_loss = test_loss/len(test_gen.dataset)
print('Test Loss: {:.6f}\n'.format(test_loss))

for i in range(num_classes):
    if class_total[i] > 0:
        print('Test Accuracy of %5s: %2d%% (%2d/%2d)' % (
            classes[i], 100 * class_correct[i] / class_total[i],
            np.sum(class_correct[i]), np.sum(class_total[i])))
    else:
        print('Test Accuracy of %5s: N/A (no training examples)' % (classes[i]))

print('\nTest Accuracy (Overall): %2d%% (%2d/%2d)' % (
    100. * np.sum(class_correct) / np.sum(class_total),
    np.sum(class_correct), np.sum(class_total)))



Test Loss: 0.498409

Test Accuracy of airplane: 85% (199/234)
Test Accuracy of automobile: 93% (229/246)
Test Accuracy of  bird: 80% (227/281)
Test Accuracy of   cat: 66% (169/253)
Test Accuracy of  deer: 87% (199/227)
Test Accuracy of   dog: 80% (201/250)
Test Accuracy of  frog: 87% (218/248)
Test Accuracy of horse: 90% (227/250)
Test Accuracy of  ship: 93% (242/258)
Test Accuracy of truck: 93% (248/265)

Test Accuracy (Overall): 85% (2159/2512)


In [0]:
# Run model on test set and save predictions

net.eval()
predictions = []

for images, _ in test_gen:
    images = images.to(device)

    output = net(images)
    _, predicted = torch.max(output, 1)
    
    img_id_start = len(predictions) + 1
    predictions += [{'image_id': img_id_start + x, 
                     'label': predicted[x].item()} for x in range(len(predicted))]

with open('caliper_cifar10_test_predictions_FangfeiLi.csv', mode='w') as preds_file:
    writer = csv.writer(preds_file, delimiter=',')
    writer.writerow(['id','label'])
    for el in predictions:
        writer.writerow([el['image_id'], el['label']])
    
files.download('caliper_cifar10_test_predictions_FangfeiLi.csv')